In [1]:
from __future__ import division
from __future__ import print_function

import datetime
import json
import sys
import time
from collections import defaultdict
import pandas as pd
import numpy as np

In [2]:
#将用户最新点击的商品作为答案
def fake_file(click_test,c):
    order = ['user_id','item_id','time']
    item_count=click_test.groupby(['user_id'],as_index=False)['user_id'].agg({'cnt':'count'})
    click_count=pd.merge(click_test,item_count,on=['user_id'],how='left')
    click_one=click_count[click_count.cnt==1]
    click_one=click_one[order]
    click_val=click_test[['user_id','time']].groupby(by='user_id',as_index=False).max()
    click_val_merge=pd.merge(click_val,click_test,on=['user_id','time'],how='left')
    click_val_merge=click_val_merge[order]
    click_test_mask=click_test.append(click_val_merge)
    click_test_mask =click_test_mask.drop_duplicates(subset=['user_id', 'item_id', 'time'],keep=False)
    click_test_mask=click_test_mask.append(click_one)
    click_test_mask.to_csv('./fake_file/fake_test_click-{}.csv'.format(c),index=False, header=None)
    click_val_merge.to_csv('./fake_file/fake_validation_click-{}.csv'.format(c),index=False, header=None)

In [3]:
#创建answer文件
def _create_answer_file_for_evaluation(answer_fname='debias_track_answer.csv'):
    train = './underexpose_train/underexpose_train_click-%d.csv'
    test = './fake_file/fake_test_click-%d.csv'

    # underexpose_test_qtime-T.csv contains only <user_id, time>
    # underexpose_test_qtime_with_answer-T.csv contains <user_id, item_id, time>
    answer = './fake_file/fake_validation_click-%d.csv'  # not released

    item_deg = defaultdict(lambda: 0)
    with open(answer_fname, 'w') as fout:
        for phase_id in range(5):
            with open(train % phase_id) as fin:
                for line in fin:
                    user_id, item_id, timestamp = line.split(',')
                    user_id, item_id, timestamp = (
                        int(user_id), int(item_id), float(timestamp))
                    item_deg[item_id] += 1
            with open(test % phase_id) as fin:
                for line in fin:
                    user_id, item_id, timestamp = line.split(',')
                    user_id, item_id, timestamp = (
                        int(user_id), int(item_id), float(timestamp))
                    item_deg[item_id] += 1
            with open(answer % phase_id) as fin:
                for line in fin:
                    user_id,item_id , timestamp = line.split(',')
                    user_id, item_id, timestamp = (
                        int(user_id), int(item_id), float(timestamp))
                    assert user_id % 11 == phase_id
                    print(phase_id, user_id, item_id, item_deg[item_id],
                          sep=',', file=fout)

In [4]:
#ndcg评估
def evaluate_each_phase(predictions, answers):
    list_item_degress = []
    for user_id in answers:
        item_id, item_degree = answers[user_id]
        list_item_degress.append(item_degree)
    list_item_degress.sort()
    median_item_degree = list_item_degress[len(list_item_degress) // 2]

    num_cases_full = 0.0
    ndcg_50_full = 0.0
    ndcg_50_half = 0.0
    num_cases_half = 0.0
    hitrate_50_full = 0.0
    hitrate_50_half = 0.0
    for user_id in answers:
        item_id, item_degree = answers[user_id]
        rank = 0
        while rank < 50 and predictions[user_id][rank] != item_id:
            rank += 1
        num_cases_full += 1.0
        if rank < 50:
            ndcg_50_full += 1.0 / np.log2(rank + 2.0)
            hitrate_50_full += 1.0
        if item_degree <= median_item_degree:
            num_cases_half += 1.0
            if rank < 50:
                ndcg_50_half += 1.0 / np.log2(rank + 2.0)
                hitrate_50_half += 1.0
    ndcg_50_full /= num_cases_full
    hitrate_50_full /= num_cases_full
    ndcg_50_half /= num_cases_half
    hitrate_50_half /= num_cases_half

    return np.array([ndcg_50_full, ndcg_50_half,
                     hitrate_50_full, hitrate_50_half], dtype=np.float32)

# submit_fname is the path to the file submitted by the participants.
# debias_track_answer.csv is the standard answer, which is not released.
def evaluate(submit_fname,
             answer_fname='debias_track_answer.csv', current_time=None):
    schedule_in_unix_time = [
        0,  # ........ 1970-01-01 08:00:00 (T=0)
        1586534399,  # 2020-04-10 23:59:59 (T=1)
        1587139199,  # 2020-04-17 23:59:59 (T=2)
        1587743999,  # 2020-04-24 23:59:59 (T=3)
        1588348799,  # 2020-05-01 23:59:59 (T=4)
        1588953599,  # 2020-05-08 23:59:59 (T=5)
        1589558399,  # 2020-05-15 23:59:59 (T=6)
        1590163199,  # 2020-05-22 23:59:59 (T=7)
        1590767999,  # 2020-05-29 23:59:59 (T=8)
        1591372799  # .2020-06-05 23:59:59 (T=9)
    ]
    assert len(schedule_in_unix_time) == 10
    for i in range(1, len(schedule_in_unix_time) - 1):
        # 604800 == one week
        assert schedule_in_unix_time[i] + 604800 == schedule_in_unix_time[i + 1]

    if current_time is None:
        current_time = int(time.time())
    print('current_time:', current_time)
    print('date_time:', datetime.datetime.fromtimestamp(current_time))
    current_phase = 0
    while (current_phase < 9) and (
            current_time > schedule_in_unix_time[current_phase + 1]):
        current_phase += 1
    print('current_phase:', current_phase)

    try:
        answers = [{} for _ in range(10)]
        with open(answer_fname, 'r') as fin:
            for line in fin:
                line = [int(x) for x in line.split(',')]
                phase_id, user_id, item_id, item_degree = line
                assert user_id % 11 == phase_id
                # exactly one test case for each user_id
                answers[phase_id][user_id] = (item_id, item_degree)
    except Exception as _:
        print( 'server-side error: answer file incorrect')

    try:
        predictions = {}
        with open(submit_fname, 'r') as fin:
            for line in fin:
                line = line.strip()
                if line == '':
                    continue
                line = line.split(',')
                user_id = int(line[0])
                if user_id in predictions:
                    print( 'submitted duplicate user_ids')
                item_ids = [int(i) for i in line[1:]]
                if len(item_ids) != 50:
                    print( 'each row need have 50 items')
                if len(set(item_ids)) != 50:
                    print( 'each row need have 50 DISTINCT items')
                predictions[user_id] = item_ids
    except Exception as _:
        print( 'submission not in correct format')

    scores = np.zeros(4, dtype=np.float32)

    # The final winning teams will be decided based on phase T=7,8,9 only.
    # We thus fix the scores to 1.0 for phase 0,1,2,...,6 at the final stage.
    if current_phase >= 7:  # if at the final stage, i.e., T=7,8,9
        scores += 7.0  # then fix the scores to 1.0 for phase 0,1,2,...,6
    phase_beg = (7 if (current_phase >= 7) else 0)
    phase_end = current_phase + 1
    for phase_id in range(phase_beg, phase_end):
        for user_id in answers[phase_id]:
            if user_id not in predictions:
                print('user_id %d of phase %d not in submission' % (
                        user_id, phase_id))
        try:
            # We sum the scores from all the phases, instead of averaging them.
            scores += evaluate_each_phase(predictions, answers[phase_id])
        except Exception as _:
            print( 'error occurred during evaluation')
    print('===============evaluation=================')
    print('score:',scores[0])
    print('hitrate_50_full:',scores[2])
    print('ndcg_50_full:',scores[0])
    print('hitrate_50_half:',scores[3])
    print('ndcg_50_half:',scores[1])
    
    '''
    return report_score(
        stdout, score=float(scores[0]),
        ndcg_50_full=float(scores[0]), ndcg_50_half=float(scores[1]),
        hitrate_50_full=float(scores[2]), hitrate_50_half=float(scores[3]))
    '''

In [6]:
#创建验证集
now_phase=4
for c in range(now_phase + 1):
    test_fname='./underexpose_test/underexpose_test_click-{}.csv'
    click_test = pd.read_csv(test_fname.format(c,c), header=None,  names=['user_id', 'item_id', 'time']) 
    fake_file(click_test,c)

In [7]:
#生成答案文件
_create_answer_file_for_evaluation(answer_fname='./fake_file/fake_debias_track_answer.csv') 

In [8]:
#!/usr/bin/env python  
# -*- coding:utf-8 -*-  
  
import pandas as pd  
from tqdm import tqdm  
from collections import defaultdict  
import math  
  
  
def get_sim_item(df, user_col, item_col, use_iif=False):  
    #把原先的click表 同用户id进行group
    user_item_ = df.groupby(user_col)[item_col].agg(set).reset_index() 
    #转换成字典  这时候是一个用户 -- {商品集合} 的倒查表
    user_item_dict = dict(zip(user_item_[user_col], user_item_[item_col]))  
    #这个list中记录了item之间的相似性
    sim_item = {}  

    item_cnt = defaultdict(int)  
    #这一段像是在计算item的同现性 用这个来近似item相似性
    #如果在整个click表中 两个item同现次数高 则 相关度/相似性高
    #从倒查表中按行取pair
    for user, items in tqdm(user_item_dict.items()):  
        #对item集合中的每一个item
        for i in items:  
            #在总的item dict中 这个item计数一次 统计item出现次数 也就是流行度
            item_cnt[i] += 1  
            #这变成了一个嵌套的list
            sim_item.setdefault(i, {})  
            for relate_item in items:  
                if i == relate_item:  
                    continue  
                sim_item[i].setdefault(relate_item, 0)  
                #是否使用use——iif 消除用户活跃的影响
                if not use_iif:  
                    sim_item[i][relate_item] += 1  
                else:  
                    sim_item[i][relate_item] += 1 / math.log(1 + len(items))  
    #消除商品流行度的影响
    sim_item_corr = sim_item.copy()  
    for i, related_items in tqdm(sim_item.items()):  
        for j, cij in related_items.items():  
            sim_item_corr[i][j] = cij/math.sqrt(item_cnt[i]*item_cnt[j])  
  
    return sim_item_corr, user_item_dict  
  
  
def recommend(sim_item_corr, user_item_dict, user_id, top_k, item_num):  
    rank = {} 
    #当前用户涉及到的items 
    interacted_items = user_item_dict[user_id] 
    #遍历该用户购买过的所有item 
    for i in interacted_items:  
        #对每一个item 从item相关表中找到最相似的前500个
        for j, wij in sorted(sim_item_corr[i].items(), reverse=True)[0:top_k]:  
            #j 代表候选item名称 wij代表相关度
            #去掉已经购买过的
            if j not in interacted_items:  
                rank.setdefault(j, 0)  
                rank[j] += wij  
    #从所有牵涉到的商品中 再找出前50个
    return sorted(rank.items(), key=lambda d: d[1], reverse=True)[:item_num]  
  
  
# fill user to 50 items  
def get_predict(df, pred_col, top_fill):  
    top_fill = [int(t) for t in top_fill.split(',')]  
    scores = [-1 * i for i in range(1, len(top_fill) + 1)]  
    ids = list(df['user_id'].unique())  
    #每个用户50个商品
    fill_df = pd.DataFrame(ids * len(top_fill), columns=['user_id'])  
    fill_df.sort_values('user_id', inplace=True)  
    fill_df['item_id'] = top_fill * len(ids)  
    fill_df[pred_col] = scores * len(ids)  
    df = df.append(fill_df)  
    df.sort_values(pred_col, ascending=False, inplace=True)  
    df = df.drop_duplicates(subset=['user_id', 'item_id'], keep='first')  
    df['rank'] = df.groupby('user_id')[pred_col].rank(method='first', ascending=False)  
    df = df[df['rank'] <= 50]  
    df = df.groupby('user_id')['item_id'].apply(lambda x: ','.join([str(i) for i in x])).str.split(',', expand=True).reset_index()  
    return df  
  
  
if __name__ == "__main__": 
    #设定phase
    now_phase = 4 
    train_path = './underexpose_train'  
    test_path = './fake_file'          
    recom_item = []  
    
    whole_click = pd.DataFrame()  
    #最外层循环 不同phase
    for c in range(now_phase + 1):  
        print('phase:', c)  
        #获取文件
        click_train = pd.read_csv(train_path + '/underexpose_train_click-{}.csv'.format(c), header=None,  names=['user_id', 'item_id', 'time'])  
        click_test = pd.read_csv(test_path + '/fake_test_click-{}.csv'.format(c), header=None,  names=['user_id', 'item_id', 'time'])  
        
        all_click = click_train.append(click_test)  
        
        whole_click = whole_click.append(all_click)  
        #计算item相似度 user——item是用户点击商品的倒查表
        item_sim_list, user_item = get_sim_item(all_click, 'user_id', 'item_id', use_iif=True)  
        #对于test数据集里的每一个user 遍历
        for i in tqdm(click_test['user_id'].unique()):  
            rank_item = recommend(item_sim_list, user_item, i, 500, 50)  
            for j in rank_item:  
                #增加三元组 'user_id', 'item_id', 'sim' 用户 商品 契合度
                recom_item.append([i, j[0], j[1]])  
    # find most popular items  最流行的items
    top50_click = whole_click['item_id'].value_counts().index[:50].values  
    top50_click = ','.join([str(i) for i in top50_click])  
  
    recom_df = pd.DataFrame(recom_item, columns=['user_id', 'item_id', 'sim'])  
    result = get_predict(recom_df, 'sim', top50_click)  
    #result = recom_df.groupby('user_id')['item_id'].apply(lambda x: ','.join([str(i) for i in x])).str.split(',', expand=True).reset_index() 
    result.to_csv('./fake_file/fake_baseline.csv', index=False, header=None)

phase: 0


100%|█████████████████████████████████████████████████████████████████████████████| 1663/1663 [00:05<00:00, 330.12it/s]


phase: 1


100%|█████████████████████████████████████████████████████████████████████████████| 1726/1726 [00:05<00:00, 311.49it/s]


phase: 2


100%|█████████████████████████████████████████████████████████████████████████████| 1690/1690 [00:07<00:00, 235.97it/s]


phase: 3


100%|█████████████████████████████████████████████████████████████████████████████| 1675/1675 [00:07<00:00, 230.16it/s]


phase: 4


100%|█████████████████████████████████████████████████████████████████████████████| 1708/1708 [00:09<00:00, 188.90it/s]


In [9]:
submit_fname='./fake_file/fake_baseline.csv'
#test_fname='./fake_file/fake_test_click-0.csv'
evaluate(submit_fname,answer_fname='./fake_file/fake_debias_track_answer.csv', current_time=None)

current_time: 1588599043
date_time: 2020-05-04 21:30:43
current_phase: 4
===============evaluation=================
score: 0.1984174
hitrate_50_full: 0.48728514
ndcg_50_full: 0.1984174
hitrate_50_half: 0.4280174
ndcg_50_half: 0.17911492
